<a href="https://colab.research.google.com/github/anuragverse/Deep-learning-/blob/main/Autoencoders_Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Input,Dropout

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file=r'/content/drive/MyDrive/Colab Notebooks/Datasets/pokemon.csv'
data=pd.read_csv(file)
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
data['Legendary'].value_counts(normalize=True)

,proportion
Legendary,
False,0.91875
True,0.08125


In [6]:
data['Legendary'].dtype

dtype('bool')

In [7]:
data.nunique()

,0
#,721
Name,800
Type 1,18
Type 2,18
Total,200
HP,94
Attack,111
Defense,103
Sp. Atk,105
Sp. Def,92


In [8]:
data.drop(['Name','Total'],axis=1, inplace=True)

In [9]:
data.isnull().sum()

,0
#,0
Type 1,0
Type 2,386
HP,0
Attack,0
Defense,0
Sp. Atk,0
Sp. Def,0
Speed,0
Generation,0


In [10]:
data['Type 2'].fillna('missing',inplace=True)

<ipython-input-10-e9f274b222d1>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Type 2'].fillna('missing',inplace=True)


In [11]:
data['Legendary']=data['Legendary'].astype(int)

In [12]:
data1=data.copy()

In [13]:
cat_cols=['Type 1','Type 2','Generation']
for col in cat_cols:
  freqs=data1[col].value_counts()
  cats=freqs.index[freqs>20]
  for cat in cats:
    name=str(col)+'_'+str(cat)
    data1[name]=(data1[col]==cat).astype(int)
  del data1[col]

In [14]:
data1.head()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Type 1_Water,Type 1_Normal,...,Type 2_Fighting,Type 2_Grass,Type 2_Fairy,Type 2_Steel,Generation_1,Generation_5,Generation_3,Generation_4,Generation_2,Generation_6
0,1,45,49,49,65,65,45,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,60,62,63,80,80,60,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,80,82,83,100,100,80,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,80,100,123,122,120,80,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,39,52,43,60,50,65,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
data.shape

(800, 11)

In [16]:
data1.shape

(800, 39)

In [17]:
x=data1.drop('Legendary',axis=1)
y=data1.Legendary
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True)
scores=cross_val_score (rf,x,y, scoring='roc_auc', cv=5)

In [18]:
scores

array([0.93956044, 0.89795918, 0.87598116, 0.86682365, 0.96912611])

In [19]:

dummy_data=pd.get_dummies (data['Type 1'], prefix='Type1')
dummy_data.shape[1]

18

In [20]:
y_type2=dummy_data

embedding_dim=3
inputs=Input(shape=(dummy_data.shape[1],))
dense1=Dense(15,activation='relu')(inputs)
embedded_output=Dense(embedding_dim)(dense1)

In [21]:
outputs=Dense(dummy_data.shape[1], activation='softmax') (embedded_output)
model=Model(inputs=inputs, outputs=outputs)

In [22]:
embedder=Model(inputs=inputs, outputs=embedded_output)

In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(dummy_data,y_type2,epochs=200,batch_size=100)

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.2471 - loss: 2.8321
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2295 - loss: 2.8110 
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2321 - loss: 2.7918 
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2419 - loss: 2.7663 
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1972 - loss: 2.7600 
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2181 - loss: 2.7283 
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2145 - loss: 2.7076 
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2409 - loss: 2.6720 
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2308 - loss: 2.6462 
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2432 - loss: 2.6172 
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2210 - loss: 2.6037 
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2416 - loss

In [24]:
low_dim_type1=pd.DataFrame(embedder.predict(dummy_data),columns=['t1_emb1','t1_emb2','t1_emb3'])
low_dim_type2=pd.DataFrame(embedder.predict(dummy_data),columns=['t2_emb1','t2_emb2','t2_emb3'])
low_dim_type2.head()

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,t2_emb1,t2_emb2,t2_emb3
0,4.152772,-3.660842,-6.203699
1,4.152772,-3.660842,-6.203699
2,4.152772,-3.660842,-6.203699
3,4.152772,-3.660842,-6.203699
4,-15.288439,1.263919,2.957936


In [25]:
data2=pd.concat([low_dim_type1,low_dim_type2,data.select_dtypes(['int64']),pd.get_dummies (data['Generation'], prefix='Generation')], axis=1)

In [26]:
scores.mean()

np.float64(0.9098901098901099)

In [27]:
x=data2.drop('Legendary',axis=1)
y=data2['Legendary']
rf=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True)
scores=cross_val_score(rf,x,y,scoring='roc_auc',cv=5)
scores.mean()

np.float64(0.9072213500784929)